### Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

from yellowbrick.regressor import PredictionError
from yellowbrick.regressor import ResidualsPlot
from yellowbrick.model_selection import RFECV
from yellowbrick.model_selection import ValidationCurve
from yellowbrick.model_selection import LearningCurve
from yellowbrick.model_selection import FeatureImportances

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.base import clone

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.feature_selection import SelectKBest, f_regression, chi2
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from tpot import TPOTRegressor

import warnings
warnings.filterwarnings('ignore')

/home/guireis/miniconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


## Data

### 100

In [2]:
eixo_100=pd.read_excel("dados/7_13_2020 - Acel 100Hz com irregularidade.xlsx",sheet_name="eixo")

In [3]:
eixo_100.drop(['X-values',
             'X-values.1',
             'X-values.2',
             'X-values.3',
             'X-values.4',
             'X-values.5',
             'X-values.6',
             'X-values.7',],axis=1,inplace=True)

In [4]:
eixo_100.columns=['y_eixo_r1_esq',
                'z_eixo_r1_esq',
                'y_eixo_r1_dir',
                'z_eixo_r1_dir',
                'y_eixo_r2_esq',
                'z_eixo_r2_esq',
                'y_eixo_r2_dir',
                'z_eixo_r2_dir']

In [5]:
triaxial_100=pd.read_excel("dados/7_13_2020 - Acel 100Hz com irregularidade.xlsx",sheet_name="triaxial")

In [6]:
triaxial_100.drop(['X-values','X-values.1','X-values.2','X-values.3','Distance'],axis=1,inplace=True)

In [7]:
triaxial_100.columns=['triaxial-x','triaxial-y','triaxial-z']

In [8]:
giro=pd.read_excel("dados/7_13_2020 - Acel 100Hz com irregularidade.xlsx",sheet_name="giro")

In [9]:
giro.drop(['X-values','X-values.1'],axis=1,inplace=True)

In [10]:
giro.columns=['roll','pitch']

In [11]:
susp_100=pd.read_excel("dados/7_13_2020 - Acel 100Hz com irregularidade.xlsx",sheet_name="mola + ace lvert + acel alt")

In [12]:
susp_100.drop(['X-values','X-values.1','X-values.2','X-values.3','X-values.4','X-values.5','X-values.6','X-values.7','X-values.8','X-values.9','X-values.10','X-values.11'],axis=1,inplace=True)

In [13]:
susp_100.columns=['z_r1_esq_real','z_r2_esq','z_r1_dir_real','z_r2_dir','s1_mola1','s1_mola2','s2_mola1','s2_mola2','y_r1_esq','y_r2_esq','y_r1_dir','y_r2_dir']

In [14]:
mola=susp_100[['s1_mola1','s1_mola2','s2_mola1','s2_mola2']]

In [15]:
susp_real_100 = susp_100[['z_r1_esq_real','z_r1_dir_real']]

In [16]:
susp_virtual_100 = susp_100[['z_r2_esq','z_r2_dir','y_r1_esq','y_r2_esq','y_r1_dir','y_r2_dir']]

In [17]:
nadal_100=pd.read_excel("dados/7_13_2020 - Nadal 100Hz com irregularidade.xlsx",sheet_name="Sheet1")

In [18]:
nadal_100.drop(['X-values','X-values.1','X-values.2','X-values.3'],axis=1,inplace=True)

In [19]:
nadal_100.columns=['nadal_r1_esq','nadal_r2_esq','nadal_r1_dir','nadal_r2_dir']

In [20]:
triaxial_100.drop(['triaxial-x'],axis=1,inplace=True)

In [21]:
tudo_100 = pd.concat([susp_real_100,susp_virtual_100,eixo_100,triaxial_100,giro,mola,nadal_100],axis=1)[4:]
tudo_100 = tudo_100.reset_index(drop=True)

In [22]:
x_virtual_100 = pd.concat([susp_virtual_100,eixo_100,giro],axis=1)[4:]
x_virtual_100 = x_virtual_100.reset_index(drop=True)


In [23]:
x_real_100 = pd.concat([susp_real_100,triaxial_100,mola],axis=1)[4:]
x_real_100=x_real_100.reset_index(drop=True)


In [24]:
x_tudo_100 = pd.concat([susp_real_100,susp_virtual_100,eixo_100,triaxial_100,giro,mola],axis=1)[4:]
x_tudo_100 = x_tudo_100.reset_index(drop=True)


In [25]:
y_nadal_1_esq_100 = nadal_100['nadal_r1_esq'][4:]
y_nadal_1_dir_100 = nadal_100['nadal_r1_dir'][4:]
y_nadal_2_esq_100 = nadal_100['nadal_r2_esq'][4:]
y_nadal_2_dir_100 = nadal_100['nadal_r2_dir'][4:]
y_nadal_1_esq_100 = y_nadal_1_esq_100.reset_index(drop=True)
y_nadal_1_dir_100 = y_nadal_1_dir_100.reset_index(drop=True)
y_nadal_2_esq_100 = y_nadal_2_esq_100.reset_index(drop=True)
y_nadal_2_dir_100 = y_nadal_2_dir_100.reset_index(drop=True)

### 1

In [26]:
eixo_1=pd.read_excel("dados/Acel_1Hz_com_irregularidade_V2.xlsx",sheet_name="eixo")

In [27]:
eixo_1.drop(['X-values',
             'X-values.1',
             'X-values.2',
             'X-values.3',
             'X-values.4',
             'X-values.5',
             'X-values.6',
             'X-values.7',],axis=1,inplace=True)

In [28]:
eixo_1.columns=['y_eixo_r1_esq',
                'z_eixo_r1_esq',
                'y_eixo_r1_dir',
                'z_eixo_r1_dir',
                'y_eixo_r2_esq',
                'z_eixo_r2_esq',
                'y_eixo_r2_dir',
                'z_eixo_r2_dir']

In [29]:
triaxial_1=pd.read_excel("dados/Acel_1Hz_com_irregularidade_V2.xlsx",sheet_name="triaxial")

In [30]:
triaxial_1.drop(['X-values','X-values.1','X-values.2','X-values.3','Distance'],axis=1,inplace=True)

In [31]:
triaxial_1.columns=['triaxial-x','triaxial-y','triaxial-z']

In [32]:
giro_1=pd.read_excel("dados/Acel_1Hz_com_irregularidade_V2.xlsx",sheet_name="giro")

In [33]:
giro_1.drop(['X-values','X-values.1'],axis=1,inplace=True)

In [34]:
giro_1.columns=['roll','pitch']

In [35]:
susp_1 =pd.read_excel("dados/Acel_1Hz_com_irregularidade_V2.xlsx",sheet_name="acel vert + lat + mola")

In [36]:
susp_1.drop(['X-values','X-values.1','X-values.2','X-values.3','X-values.4','X-values.5','X-values.6','X-values.7','X-values.8','X-values.9','X-values.10','X-values.11'],axis=1,inplace=True)

In [37]:
susp_1.columns=['z_r1_esq_real','z_r2_esq','z_r1_dir_real','z_r2_dir','s1_mola1','s1_mola2','s2_mola1','s2_mola2','y_r1_esq','y_r2_esq','y_r1_dir','y_r2_dir']

In [38]:
mola_1=susp_1[['s1_mola1','s1_mola2','s2_mola1','s2_mola2']]

In [39]:
susp_real_1 = susp_1[['z_r1_esq_real','z_r1_dir_real']]

In [40]:
susp_virtual_1 = susp_1[['z_r2_esq','z_r2_dir','y_r1_esq','y_r2_esq','y_r1_dir','y_r2_dir']]

In [41]:
nadal_1=pd.read_excel("dados/Nadal_1Hz_com_irregularidade_V2.xlsx",sheet_name="Sheet1")

In [42]:
nadal_1.drop(['X-values','X-values.1','X-values.2','X-values.3'],axis=1,inplace=True)

In [43]:
nadal_1.columns=['nadal_r1_esq','nadal_r2_esq','nadal_r1_dir','nadal_r2_dir']

In [44]:
triaxial_1.drop(['triaxial-x'],axis=1,inplace=True)

In [45]:
tudo_1 = pd.concat([susp_real_1,susp_virtual_1,eixo_1,triaxial_1,giro_1,mola_1,nadal_1],axis=1)[4:]
tudo_1 = tudo_1.reset_index(drop=True)

In [46]:
x_virtual_1 = pd.concat([susp_virtual_1,eixo_1,giro_1],axis=1)[4:]
x_virtual_1 = x_virtual_1.reset_index(drop=True)

In [47]:
x_real_1 = pd.concat([susp_real_1,triaxial_1,mola_1],axis=1)[4:]
x_real_1=x_real_1.reset_index(drop=True)

In [48]:
x_tudo_1 = pd.concat([susp_real_1,susp_virtual_1,eixo_1,triaxial_1,giro_1,mola_1],axis=1)[4:]
x_tudo_1 = x_tudo_1.reset_index(drop=True)

In [49]:
y_nadal_1_esq_1 = nadal_1['nadal_r1_esq'][4:]
y_nadal_1_dir_1 = nadal_1['nadal_r1_dir'][4:]
y_nadal_2_esq_1 = nadal_1['nadal_r2_esq'][4:]
y_nadal_2_dir_1 = nadal_1['nadal_r2_dir'][4:]
y_nadal_1_esq_1 = y_nadal_1_esq_1.reset_index(drop=True)
y_nadal_1_dir_1 = y_nadal_1_dir_1.reset_index(drop=True)
y_nadal_2_esq_1 = y_nadal_2_esq_1.reset_index(drop=True)
y_nadal_2_dir_1 = y_nadal_2_dir_1.reset_index(drop=True)

### 100 sem

In [50]:
eixo_100_no=pd.read_excel("dados/7_13_2020 - Acel 100Hz sem irregularidade.xlsx",sheet_name="eixo")

In [51]:
eixo_100_no.drop(['X-values',
             'X-values.1',
             'X-values.2',
             'X-values.3',
             'X-values.4',
             'X-values.5',
             'X-values.6',
             'X-values.7',],axis=1,inplace=True)

In [52]:
eixo_100_no.columns=['y_eixo_r1_esq',
                'z_eixo_r1_esq',
                'y_eixo_r1_dir',
                'z_eixo_r1_dir',
                'y_eixo_r2_esq',
                'z_eixo_r2_esq',
                'y_eixo_r2_dir',
                'z_eixo_r2_dir']

In [53]:
triaxial_100_no=pd.read_excel("dados/7_13_2020 - Acel 100Hz sem irregularidade.xlsx",sheet_name="triaxial")

In [54]:
triaxial_100_no.drop(['X-values','X-values.1','X-values.2','X-values.3','Distance'],axis=1,inplace=True)

In [55]:
triaxial_100_no.columns=['triaxial-x','triaxial-y','triaxial-z']

In [56]:
giro_100_no=pd.read_excel("dados/7_13_2020 - Acel 100Hz sem irregularidade.xlsx",sheet_name="giro")

In [57]:
giro_100_no.drop(['X-values','X-values.1'],axis=1,inplace=True)

In [58]:
giro_100_no.columns=['roll','pitch']

In [59]:
susp_100_no =pd.read_excel("dados/7_13_2020 - Acel 100Hz sem irregularidade.xlsx",sheet_name="mola + lat + vert travessa")

In [60]:
susp_100_no.drop(['X-values','X-values.1','X-values.2','X-values.3','X-values.4','X-values.5','X-values.6','X-values.7','X-values.8','X-values.9','X-values.10','X-values.11'],axis=1,inplace=True)

In [61]:
susp_100_no.columns=['z_r1_esq_real','z_r2_esq','z_r1_dir_real','z_r2_dir','s1_mola1','s1_mola2','s2_mola1','s2_mola2','y_r1_esq','y_r2_esq','y_r1_dir','y_r2_dir']

In [62]:
mola_100_no=susp_100_no[['s1_mola1','s1_mola2','s2_mola1','s2_mola2']]

In [63]:
susp_real_100_no = susp_100_no[['z_r1_esq_real','z_r1_dir_real']]

In [64]:
susp_virtual_100_no = susp_100_no[['z_r2_esq','z_r2_dir','y_r1_esq','y_r2_esq','y_r1_dir','y_r2_dir']]

In [65]:
nadal_100_no=pd.read_excel("dados/7_13_2020 - Nadal 100Hz sem irregularidades.xlsx",sheet_name="Sheet1")

In [66]:
nadal_100_no.drop(['X-values','X-values.1','X-values.2','X-values.3'],axis=1,inplace=True)

In [67]:
nadal_100_no.columns=['nadal_r1_esq','nadal_r2_esq','nadal_r1_dir','nadal_r2_dir']

In [68]:
triaxial_100_no.drop(['triaxial-x'],axis=1,inplace=True)

In [69]:
tudo_100_no = pd.concat([susp_real_100_no,susp_virtual_100_no,eixo_100_no,triaxial_100_no,giro_100_no,mola_100_no,nadal_100_no],axis=1)[4:]
tudo_100_no = tudo_100_no.reset_index(drop=True)

In [70]:
x_virtual_100_no = pd.concat([susp_virtual_100_no,eixo_100_no,giro_100_no],axis=1)[4:]
x_virtual_100_no = x_virtual_100_no.reset_index(drop=True)

In [71]:
x_real_100_no= pd.concat([susp_real_100_no,triaxial_100_no,mola_100_no],axis=1)[4:]
x_real_100_no=x_real_100_no.reset_index(drop=True)

In [72]:
x_tudo_100_no = pd.concat([susp_real_100_no,susp_virtual_100_no,eixo_100_no,triaxial_100_no,giro_100_no,mola_100_no],axis=1)[4:]
x_tudo_100_no = x_tudo_100_no.reset_index(drop=True)

In [73]:
y_nadal_1_esq_100_no = nadal_100_no['nadal_r1_esq'][4:]
y_nadal_1_dir_100_no = nadal_100_no['nadal_r1_dir'][4:]
y_nadal_2_esq_100_no = nadal_100_no['nadal_r2_esq'][4:]
y_nadal_2_dir_100_no = nadal_100_no['nadal_r2_dir'][4:]
y_nadal_1_esq_100_no = y_nadal_1_esq_100_no.reset_index(drop=True)
y_nadal_1_dir_100_no = y_nadal_1_dir_100_no.reset_index(drop=True)
y_nadal_2_esq_100_no = y_nadal_2_esq_100_no.reset_index(drop=True)
y_nadal_2_dir_100_no= y_nadal_2_dir_100_no.reset_index(drop=True)

In [74]:
y_nadal_1_esq_100_no.size

36852

## Definições

In [75]:
def residuo(model,X_train,X_test,y_train,y_test):
    plt.figure(figsize=(13,8))
    visualizer = ResidualsPlot(model,hist=True)
    visualizer.fit(X_train[0:1000], y_train.squeeze()[0:1000])  # Fit the training data to the visualizer
    visualizer.score(X_test[0:1000], y_test.squeeze()[0:1000])  # Evaluate the model on the test data
    return visualizer.show()

In [76]:
def feat_importances(model,train):
    feat_importances = pd.Series(model.feature_importances_, index= train.columns)
    return feat_importances.nlargest(20).plot(kind='barh',color='#cf6275')

In [77]:
def feature_importance(model,X_train,y_train):
    plt.figure(figsize=(10,10))
    labels = list(map(lambda s: s.title(), X_train.columns))
    viz = FeatureImportances(model, labels=labels, relative=False, colormap='Blues')
    viz.fit(X_train, y_train)
    return viz.show()

In [78]:
def metric(y_test,predict):
    mse = mean_squared_error(y_test, predict)
    mae= mean_absolute_error(y_test,predict)
    r2= r2_score(y_test,predict)
    return print("MSE:{}".format(mse),"\nMAE:{}".format(mae),"\nR2:{}".format(r2))

In [81]:
def tpot(X_train,X_test,y_train,y_test,nome):
    pipeline_optimizer = TPOTRegressor(generations=5, population_size=50, cv=5, random_state=42, verbosity=2)
    pipeline_optimizer.fit(X_train, y_train)
    predict=pipeline_optimizer.predict(X_test)
    pipeline_optimizer.export(nome)
    return (print(pipeline_optimizer.score(X_test, y_test)),"\n",metric(y_test,predict))

## 100 hz TPOT

### Nadal_1_esq

In [88]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_100,y_nadal_1_esq_100,test_size=0.3)

In [89]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_100,y_nadal_1_esq_100,test_size=0.3)

In [90]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo,'nadal_1_esq_tudo')

Optimization Progress:  85%|████████▍ | 254/300 [4:24:37<1:06:45, 87.07s/pipeline] 
Optimization Progress: 305pipeline [5:47:54, 89.24s/pipeline]                      
                                                             8908e-05
Best pipeline: RandomForestRegressor(ExtraTreesRegressor(SGDRegressor(input_matrix, alpha=0.0, eta0=0.01, fit_intercept=True, l1_ratio=0.25, learning_rate=invscaling, loss=squared_loss, penalty=elasticnet, power_t=0.1), bootstrap=False, max_features=0.8, min_samples_leaf=5, min_samples_split=17, n_estimators=100), bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=13, n_estimators=100)
-4.455355773712198e-05
MSE:4.455355773712198e-05 
MAE:0.0044772679593507265 
R2:0.986013539017378


(None, '\n', None)

In [91]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_1_esq_real')

Optimization Progress:  84%|████████▎ | 251/300 [1:26:30<25:34, 31.32s/pipeline]   
Optimization Progress: 301pipeline [1:52:15, 25.64s/pipeline]                   
                                                             825228294
Best pipeline: XGBRegressor(ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=1.0, min_samples_leaf=3, min_samples_split=2, n_estimators=100), learning_rate=0.1, max_depth=7, min_child_weight=17, n_estimators=100, nthread=1, objective=reg:squarederror, subsample=0.2)
-8.748850034792713e-05
MSE:8.748850034792713e-05 
MAE:0.005992783302455567 
R2:0.9721311928847856


(None, '\n', None)

### Nadal 1 dir

In [85]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_100,y_nadal_1_dir_100,test_size=0.3)

In [86]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_100,y_nadal_1_dir_100,test_size=0.3)

In [87]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo,'nadal_1_dir_tudo')

Optimization Progress:  84%|████████▍ | 252/300 [3:41:15<1:18:52, 98.59s/pipeline] 
Optimization Progress: 304pipeline [5:15:33, 136.55s/pipeline]                     
                                                              247e-05
Best pipeline: XGBRegressor(RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.8, min_samples_leaf=2, min_samples_split=16, n_estimators=100), learning_rate=0.5, max_depth=4, min_child_weight=3, n_estimators=100, nthread=1, objective=reg:squarederror, subsample=0.45)
-4.6319502286662825e-05
MSE:4.6319502286662825e-05 
MAE:0.004534263834557969 
R2:0.9852134310911124


(None, '\n', None)

In [88]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_1_dir_real')

Optimization Progress: 100%|██████████| 300/300 [2:01:19<00:00, 59.18s/pipeline]
                                                                                
Best pipeline: XGBRegressor(RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.8, min_samples_leaf=2, min_samples_split=6, n_estimators=100), learning_rate=0.1, max_depth=6, min_child_weight=15, n_estimators=100, nthread=1, objective=reg:squarederror, subsample=0.35000000000000003)
-9.061049067611083e-05
MSE:9.061049067611083e-05 
MAE:0.005661507077853518 
R2:0.9712870893367224


(None, '\n', None)

### Nadal_2_esq

In [89]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_100,y_nadal_2_esq_100,test_size=0.3)

In [90]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_100,y_nadal_2_esq_100,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo,'nadal_2_esq_tudo')

Optimization Progress:  37%|███▋      | 111/300 [1:09:32<2:25:40, 46.25s/pipeline]

In [ ]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_2_esq_real')

### Nadal_2_dir

In [ ]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_100,y_nadal_2_dir_100,test_size=0.3)

In [ ]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_100,y_nadal_2_dir_100,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo,'nadal_2_dir_tudo')

In [ ]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_2_dir_real')

## 1Hz Tpot

### Nadal_1_esq

In [161]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_1,y_nadal_1_esq_1,test_size=0.3)

In [162]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_1,y_nadal_1_esq_1,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_rudo,'nadal_1_esq_tudo_1')

In [165]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_1_esq_real_1')

                                                                            
                                                                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.
                                                                            
Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=9, n_estimators=100)
-6.780401713721122e-05
MSE:6.780401713721122e-05 
MAE:0.0038441531378675792 
R2:0.9786980768245297


(None, '\n', None)

### Nadal 1 dir

In [168]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_1,y_nadal_1_dir_1,test_size=0.3)

In [84]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_1,y_nadal_1_dir_1,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_rudo,'nadal_1_dir_tudo_1')

In [165]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_1_dir_real_1')

                                                                            
                                                                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.
                                                                            
Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=9, n_estimators=100)
-6.780401713721122e-05
MSE:6.780401713721122e-05 
MAE:0.0038441531378675792 
R2:0.9786980768245297


(None, '\n', None)

### Nadal_2_esq

In [173]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_1,y_nadal_2_esq_1,test_size=0.3)

In [174]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_1,y_nadal_2_esq_1,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_rudo,'nadal_2_esq_tudo_1')

In [165]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_2_esq_real_1')

                                                                            
                                                                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.
                                                                            
Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=9, n_estimators=100)
-6.780401713721122e-05
MSE:6.780401713721122e-05 
MAE:0.0038441531378675792 
R2:0.9786980768245297


(None, '\n', None)

### Nadal_2_dir

In [178]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_1,y_nadal_2_dir_1,test_size=0.3)

In [179]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_1,y_nadal_2_dir_1,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_rudo,'nadal_2_dir_tudo_1')

In [182]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_2_dir_real_1')

                                                                    
                                                                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.


RuntimeError: A pipeline has not yet been optimized. Please call fit() first.

## 100Hz sem Tpot

### Nadal_1_esq

In [161]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_100_no,y_nadal_1_esq_100_no,test_size=0.3)

In [162]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_100_no,y_nadal_1_esq_100_no,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_rudo,'nadal_1_esq_tudo_100_no')

In [165]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_1_esq_real_100_no')

                                                                            
                                                                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.
                                                                            
Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=9, n_estimators=100)
-6.780401713721122e-05
MSE:6.780401713721122e-05 
MAE:0.0038441531378675792 
R2:0.9786980768245297


(None, '\n', None)

### Nadal 1 dir

In [168]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_100_no,y_nadal_1_dir_100_no,test_size=0.3)

In [169]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_100_no,y_nadal_1_dir_100_no,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_rudo,'nadal_1_dir_tudo_100_no')

In [165]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_1_dir_real_100_no')

                                                                            
                                                                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.
                                                                            
Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=9, n_estimators=100)
-6.780401713721122e-05
MSE:6.780401713721122e-05 
MAE:0.0038441531378675792 
R2:0.9786980768245297


(None, '\n', None)

### Nadal_2_esq

In [173]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_100_no,y_nadal_2_esq_100_no,test_size=0.3)

In [174]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_100_no,y_nadal_2_esq_100_no,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_rudo,'nadal_2_esq_tudo_100_no')

In [165]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_2_esq_real_100_no')

                                                                            
                                                                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.
                                                                            
Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=9, n_estimators=100)
-6.780401713721122e-05
MSE:6.780401713721122e-05 
MAE:0.0038441531378675792 
R2:0.9786980768245297


(None, '\n', None)

### Nadal_2_dir

In [178]:
X_train_tudo,X_test_tudo,y_train_tudo,y_test_tudo=train_test_split(x_tudo_100_no,y_nadal_2_dir_100_no,test_size=0.3)

In [179]:
X_train_real,X_test_real,y_train_real,y_test_real=train_test_split(x_real_100_no,y_nadal_2_dir_100_no,test_size=0.3)

In [ ]:
tpot(X_train_tudo,X_test_tudo,y_train_tudo,y_test_rudo,'nadal_2_dir_tudo_100_no')

In [182]:
tpot(X_train_real,X_test_real,y_train_real,y_test_real,'nadal_2_dir_real_100_no')

                                                                    
                                                                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.


RuntimeError: A pipeline has not yet been optimized. Please call fit() first.